In [1]:
pip install PyPDF2 openai pandas

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
   ---------------------------------------- 0.0/735.8 kB ? eta -:--:--
   ---------------------------------------- 735.8/735.8 kB 5.1 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------------- -- 1.8/2.0 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 6.8 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached annotated_types-0.7.0-py3-none


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import openai
import json
from PyPDF2 import PdfReader
import re
import pandas as pd
import os

In [3]:
# Canonical headers and flexible matching patterns
SECTION_HEADERS = {
    "Key Points and Weather Watch": r"key points and weather watch",
    "Change in Emphasis:": r"change in emphasis",
    "Key Points": r"key points",
    "Weather Summaries": r"weather summaries",
    "Confidence": r"confidence",
    "Modifications": r"modifications",
    "Modified Fields": r"modified fields",
    "Evolution and Discussion :": r"(evolution and discussion|part b)",
    "Precipitation :": r"precipitation",
    "Wind :": r"wind",
    "Temperature :": r"temperature",
    "Low Cloud / Visibility :": r"low cloud\s*/\s*visibility",
    "Other / Miscellaneous :": r"other\s*/\s*miscellaneous"
}

# Safe group names for regex
def safe_group_name(header):
    return re.sub(r'[^a-zA-Z0-9_]', '_', header)

# Match headers even if inline
HEADER_PATTERN = re.compile(
    r"(?:^|\n+)(?P<header>" + "|".join(f"(?P<{safe_group_name(k)}>{v})" for k, v in SECTION_HEADERS.items()) + r")\s*[:\-]?",
    re.IGNORECASE
)


# Extract all text
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            full_text += page_text + "\n"
    return full_text

# Split into sections using inline-aware headers
def split_into_sections(text):
    section_dict = {header: "" for header in SECTION_HEADERS}
    matches = list(HEADER_PATTERN.finditer(text))

    if not matches:
        return section_dict

    # Add pre-header text to first matched section
    intro = text[:matches[0].start()].strip()
    for canon, pattern in SECTION_HEADERS.items():
        if re.fullmatch(pattern, matches[0].group("header").strip(), re.IGNORECASE):
            section_dict[canon] = intro
            break

    for i, match in enumerate(matches):
        header_text = match.group("header").strip().lower()

        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        section_body = text[start:end].strip()

        for canon, pattern in SECTION_HEADERS.items():
            if re.fullmatch(pattern, header_text, re.IGNORECASE):
                section_dict[canon] = section_body
                break

    return section_dict

# Pipeline
def parse_structured_pdf(pdf_path):
    raw_text = extract_text_from_pdf(pdf_path)
    return split_into_sections(raw_text)


In [30]:
# Create the dataframe
structured_report = parse_structured_pdf(r"C:\Users\hp\OneDrive - Université Mohammed VI Polytechnique\Documents\University_of_Exeter\Studies\Events\BEI\2024\mae.202401010436.pdf")
df = pd.DataFrame([structured_report])
df.head()

Key Points and Weather Watch  \
0                                

                                 Change in Emphasis:  \
0  Nil. \n \nToday: Early showers becoming confin...   

                                          Key Points Weather Summaries  \
0   Further spells of wet weather following acro...                     

                                          Confidence  \
0   High for main th emes but track of Tuesday’s...   

                                       Modifications Modified Fields  \
0   Based on GM/UKV blend to early Tuesday then,...                   

                          Evolution and Discussion :  \
0  – Evolution :   \nE’ward passage of a transien...   

                                     Precipitation :  \
0   Waving frontal zone continues to linger over...   

                                              Wind :  \
0   This afternoon/evening, turning windier in m...   

                                       Temperature : Low Cloud / Visibility :  \
0   Today, maxima higher than yesterday for much...                            

  Other / Miscellaneous :  
0

In [20]:
# Replace with the path to your folder
folder_path = "C:\\Users\\hp\\OneDrive - Université Mohammed VI Polytechnique\\Documents\\University_of_Exeter\\Studies\\Events\\BEI\\2024"
pdf_names = []

# Loop through all PDF files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        full_path = os.path.join(folder_path, filename)
        # Call your existing extraction function
        pdf_names.append(full_path)


In [22]:
len(pdf_names)

1370

In [52]:
numerical_parts_int = [int(re.sub(r"\D", "", name[-16:-4])) for name in pdf_names]

In [31]:
from pathlib import Path
import pandas as pd

df = pd.DataFrame()
failed_files = []

for name in pdf_names:
    try:
        structured_report = parse_structured_pdf(name)
        df = pd.concat([df, pd.DataFrame([structured_report])], ignore_index=True)
    except Exception as e:
        print(f"Failed to parse {name}: {e}")
        failed_files.append(name)


Failed to parse C:\Users\hp\OneDrive - Université Mohammed VI Polytechnique\Documents\University_of_Exeter\Studies\Events\BEI\2024\mae.202401211611.pdf: EOF marker not found


In [46]:
failed_date = int(re.sub(r"\D", "", failed_files[0][-16:-4]))

In [47]:
len(numerical_parts_int)

1370

In [54]:
numerical_parts_int.remove(failed_date)

In [55]:
len(numerical_parts_int)

1369

In [56]:
len(df)

1369

In [57]:
df["numeric_id"] = numerical_parts_int

In [ ]:
df["datetime"] = pd.to_datetime(df["numeric_id"].astype(str), format="%Y%m%d%H%M")

Key Points and Weather Watch  \
0                                                         
1                                                         
2                                                         
3                                                         
4                                                         
...                                                 ...   
1364   Further h eavy rain for parts of Scotland an...   
1365   Today, further h eavy rain for parts of Scot...   
1366   Today, further h eavy rain for parts of Scot...   
1367   NSWWS rain/snow warnings in place for N/NW p...   
1368   A number of NSWWS warnings in force for the ...   

                                    Change in Emphasis:  \
0     Nil. \n \nToday: Early showers becoming confin...   
1     Part A update  \n \nToday: Early showers becom...   
2     Part A update  \n \nToday: Early showers becom...   
3     Nil. \n \nTonight:  Clear in Scotland at first...   
4     Nil. \n \nTuesday : Early cloud and rain over ...   
...                                                 ...   
1364                                               Nil.   
1365                                               Nil.   
1366  Nil. \n \nWeat her Summaries  \n \nToday : Ear...   
1367                                               Nil.   
1368                                                      

                                             Key Points  \
0      Further spells of wet weather following acro...   
1      Unsettled focus until end of week, ahead of ...   
2      Unsettled focus until end of week, ahead of ...   
3      Unsettled focus until end of week, ahead of ...   
4      Unsettled focus until end of week, ahead of ...   
...                                                 ...   
1364                                                      
1365                                                      
1366                                                      
1367                                                      
1368                                                      

                                      Weather Summaries  \
0                                                         
1                                                         
2                                                         
3                                                         
4                                                         
...                                                 ...   
1364  Over night:  Rain and snow, often heavy, acros...   
1365  Today : Early rain Scotland/NI (and snow far N...   
1366                                                      
1367  Tonight:  Band of rain/sleet/hill snow N Scotl...   
1368  Over night:  Band rain/sleet/hill snow N Scotl...   

                                             Confidence  \
0      High for main th emes but track of Tuesday’s...   
1      High for main themes but track of Tuesday’s ...   
2      High for main themes but track of Tuesday’s ...   
3                                                         
4      High for main themes but track of Tuesday’s ...   
...                                                 ...   
1364   High for overall evolution and main themes ....   
1365   High for overall evolution and main themes. ...   
1366   High for overall evolution and main themes. ...   
1367   High for overall evolution and main themes. ...   
1368   High for overall evolution and main themes. ...   

                                          Modifications  \
0      Based on GM/UKV blend to early Tuesday then,...   
1      Based on GM/UKV blend to early Tuesday.  \n...   
2      Based on GM/UKV blend to early Tuesday.  \n...   
3      Based heavily o n 12Z EC for synoptic evolut...   
4                                                         
...                                                 ...   
1364   Based on merge of UKV/GM.  \n \n \n \n \n© C...   
1365   Ba

In [59]:
df.columns

Index(['Key Points and Weather Watch', 'Change in Emphasis:', 'Key Points',
       'Weather Summaries', 'Confidence', 'Modifications', 'Modified Fields',
       'Evolution and Discussion :', 'Precipitation :', 'Wind :',
       'Temperature :', 'Low Cloud / Visibility :', 'Other / Miscellaneous :',
       'numeric_id', 'datetime'],
      dtype='object')

In [60]:
df.columns = [
    'Key Points and Weather Watch', 'Change in Emphasis', 'Key Points',
    'Weather Summaries', 'Confidence', 'Modifications', 'Modified Fields',
    'Evolution and Discussion', 'Precipitation', 'Wind',
    'Temperature', 'Low Cloud / Visibility', 'Other / Miscellaneous',
    'numeric_id', 'datetime'
]

In [68]:
df2 = df.copy()

In [ ]:
df2['Key Points and Weather Watch'] = df2['Key Points and Weather Watch'].fillna('') + ' ' + df2['Key Points'].fillna('')
df2['Key Points and Weather Watch'] = df2['Key Points and Weather Watch'].str.strip()
df2['Change in Emphasis'] = df2['Change in Emphasis'].fillna('') + ' ' + df2['Weather Summaries'].fillna('')
df2['Change in Emphasis'] = df2['Change in Emphasis'].str.strip()

In [71]:

df2.drop(columns=['Key Points', 'Weather Summaries'], inplace=True)

In [73]:
df3 = df2.copy()

In [74]:
df3['end_datetime'] = df3['datetime'] + pd.Timedelta(days=1)

In [76]:
df3.to_csv("pdf_meteorological_2024.csv", index=False)

In [77]:
df3

Key Points and Weather Watch  \
0      Further spells of wet weather following acro...   
1      Unsettled focus until end of week, ahead of ...   
2      Unsettled focus until end of week, ahead of ...   
3      Unsettled focus until end of week, ahead of ...   
4      Unsettled focus until end of week, ahead of ...   
...                                                 ...   
1364   Further h eavy rain for parts of Scotland an...   
1365   Today, further h eavy rain for parts of Scot...   
1366   Today, further h eavy rain for parts of Scot...   
1367   NSWWS rain/snow warnings in place for N/NW p...   
1368   A number of NSWWS warnings in force for the ...   

                                     Change in Emphasis  \
0     Nil. \n \nToday: Early showers becoming confin...   
1     Part A update  \n \nToday: Early showers becom...   
2     Part A update  \n \nToday: Early showers becom...   
3     Nil. \n \nTonight:  Clear in Scotland at first...   
4     Nil. \n \nTuesday : Early cloud and rain over ...   
...                                                 ...   
1364  Nil. Over night:  Rain and snow, often heavy, ...   
1365  Nil. Today : Early rain Scotland/NI (and snow ...   
1366  Nil. \n \nWeat her Summaries  \n \nToday : Ear...   
1367  Nil. Tonight:  Band of rain/sleet/hill snow N ...   
1368  Over night:  Band rain/sleet/hill snow N Scotl...   

                                             Confidence  \
0      High for main th emes but track of Tuesday’s...   
1      High for main themes but track of Tuesday’s ...   
2      High for main themes but track of Tuesday’s ...   
3                                                         
4      High for main themes but track of Tuesday’s ...   
...                                                 ...   
1364   High for overall evolution and main themes ....   
1365   High for overall evolution and main themes. ...   
1366   High for overall evolution and main themes. ...   
1367   High for overall evolution and main themes. ...   
1368   High for overall evolution and main themes. ...   

                                          Modifications  \
0      Based on GM/UKV blend to early Tuesday then,...   
1      Based on GM/UKV blend to early Tuesday.  \n...   
2      Based on GM/UKV blend to early Tuesday.  \n...   
3      Based heavily o n 12Z EC for synoptic evolut...   
4                                                         
...                                                 ...   
1364   Based on merge of UKV/GM.  \n \n \n \n \n© C...   
1365   Based on merge of UKV/GM.  \n \n \n \n \n© C...   
1366   Based on 80/20 merge of 03Z UKV and 00Z GM. ...   
1367   Based on 80/20 merge of 09Z UKV and 06Z GM. ...   
1368   Based on 80/20 merge of UKV and GM.  \n All...   

                                        Modified Fields  \
0                                                         
1                                                         
2                                                         
3                                                         
4      Modified PMSL, WBFL and rain/snow discrimination   
...                                                 ...   
1364  © Crown Copyright 2024 , Met Office  \n \n \n ...   
1365                                                      
1366  © Crown Copyright 2024 , Met Office  \n \n \n ...   
1367  © Crown Copyr ight 2024 , Met Office  \n \n \n...   
1368  © Crown Copyright 2024 , Met Office   \n \n \n...   

                               Evolution and Discussion  \
0     – Evolution :   \nE’ward passage of a transien...   
1     – Evolution :   \nE’ward passage of a transien...   
2     – Evolution :   \n \nE’ward passage of a trans...   
3     – Evolution :   \n \n A complex multi -structu...   
4     – Evolution :   \n \n Occluding frontal zone w...   
...                                                 ...   
1364  Evolution and Discussion :   \nDepression curr...   
1365  Evol